This Jupyter notebook should provide a quick overview of the dataset and its files.
The ideal setup would be to load the data for the project over to AWS S3 buckets, while this notebook runs on a AWS EMR cluster.

The notebook will show as well some assumptions used on the different datasets to later perform some QA.

Let begin with some basic settings and imports:

In [1]:
import configparser
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType, TimestampType, \
    StructType, StructField, DoubleType, LongType

Initializa Spark session:

In [2]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

The following sections defines the files addresses in the repository:

In [3]:
input_data = '../data'
customerlogins_file = os.path.join(input_data, 'customerlogins.csv')
customerregistration_files = os.path.join(input_data, 'customerregistration*.csv')
instantgames_file = os.path.join(input_data, 'instantgamespurchases.csv')
lotterygames_file = os.path.join(input_data, 'lotterygamespurchases.csv')

### Customer login data overview:

Import the data and show the first 10 rows:

In [4]:
df = spark.read.csv(customerlogins_file, header=True, sep=';')

In [6]:
df.show(n=10)

+--------------------+--------+--------------+
|           timestamp|    site|customernumber|
+--------------------+--------+--------------+
|2018-05-04 16:51:...|websiteF|        565560|
|2020-01-29 09:17:...|websiteZ|        566115|
|2019-02-16 02:20:...|websiteX|        564512|
|2019-04-11 18:36:...|websiteZ|        565588|
|2018-06-07 13:47:...|websiteE|        565823|
|2019-11-07 06:47:...|websiteX|        565719|
|2019-05-25 02:55:...|websiteB|        565672|
|2020-01-09 14:18:...|websiteF|        566021|
|2018-06-02 06:55:...|websiteZ|        565540|
|2019-05-18 21:28:...|websiteG|        565659|
+--------------------+--------+--------------+
only showing top 10 rows



In [7]:
print(f"Dataframe shape: ({df.count()}, {len(df.columns)})")
print(f"Records with empty timestamp value: {df.where(col('timestamp').isNull()).count()}")
print(f"Records with empty site value: {df.where(col('site').isNull()).count()}")
print(f"Records with empty customernumber value: {df.where(col('customernumber').isNull()).count()}")
print(f"Records with digits inside site: {df.select('site').distinct().where(col('site').rlike('^[0-9]*$')).count()}")
print(f"Records with literals inside customernumber: {df.select('customernumber').distinct().where(~col('customernumber').rlike('^[0-9]*$')).count()}")

Dataframe shape: (9515715, 3)
Records with empty timestamp value: 0
Records with empty site value: 100043
Records with empty customernumber value: 100395
Records with digits inside site: 0
Records with literals inside customernumber: 0


### Customer registration data overview:

Import the data and show the first 10 rows:

In [8]:
df = spark.read.csv(customerregistration_files, header=True, sep=';')

In [9]:
df.show(n=10)

+--------------------+--------+--------------------+--------------------+----------------+----------------+--------------------------+--------------------+---------------------------+-------------------------+-----------------------------+---------------------+--------------------+--------------+
|           timestamp|    site|       customeremail|         dateofbirth|      familyname|      givennames|primaryaddress_addressline| primaryaddress_city|primaryaddress_federalstate|primaryaddress_postalcode|primaryaddress_sovereignstate|primaryaddress_street|    registrationdate|customernumber|
+--------------------+--------+--------------------+--------------------+----------------+----------------+--------------------------+--------------------+---------------------------+-------------------------+-----------------------------+---------------------+--------------------+--------------+
|2018-06-01 23:01:...|websiteH|phjiwbc15@ifcgiu.net|1983-05-17 18:26:...|Jonathon Daniels|     Betsy Hogan

In [10]:
print(f"Dataframe shape: ({df.count()}, {len(df.columns)})")
not_null_cols = ['timestamp', 'site', 'customeremail', 'familyname',
                 'givennames', 'customernumber', 'dateofbirth']
for c in not_null_cols:
    print(f"Records with empty {c} value: {df.where(col(c).isNull()).count()}")

invalid_email_records = df.select('customeremail').where(~col('customeremail').rlike('^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$')).count()
print(f"Records with invalid email values: {invalid_email_records}")
print(f"Records with duplicate customernumber value: {df.select('customernumber').count() - df.select('customernumber').drop_duplicates().count()}")

Dataframe shape: (1000214, 14)
Records with empty timestamp value: 0
Records with empty site value: 10146
Records with empty customeremail value: 9984
Records with empty familyname value: 10063
Records with empty givennames value: 10034
Records with empty customernumber value: 1
Records with empty dateofbirth value: 10112
Records with invalid email values: 595351
Records with duplicate customernumber value: 214


### Instant games data overview:

Import the data and show the first 10 rows:

In [11]:
df = spark.read.csv(instantgames_file, header=True, sep=';')

In [12]:
df.show(n=10)

+--------------------+--------+--------------+--------+--------------+--------------------+-----------------+----------+-------------------+----------------+-------------+
|           timestamp| sitetid|customernumber|currency|aggregationkey|            gamename|highfrequencygame|priceineur|           feeineur|ticketexternalid|winningsineur|
+--------------------+--------+--------------+--------+--------------+--------------------+-----------------+----------+-------------------+----------------+-------------+
|2018-11-13 06:30:...|websiteH|        667124|     eur|          0716|  gslsngoldenesieben|             null|       2.0|0.40000000000000002|            GOV7|         null|
|2019-01-17 00:55:...|websiteZ|        666294|     eur|          0148|             Nikolos|             null|       3.0|0.80000000000000004|        HUC49302|         null|
|2019-11-06 22:32:...|websiteD|        667578|     eur|          0724|  gslsngoldhoernchen|             null|       2.0|0.40000000000000002|

In [13]:
print(f"Dataframe shape: ({df.count()}, {len(df.columns)})")
not_null_cols =  ['timestamp', 'sitetid', 'customernumber', 'gamename',
                  'priceineur', 'feeineur', 'ticketexternalid']
for c in not_null_cols:
    print(f"Records with empty {c} value: {df.where(col(c).isNull()).count()}")

not_neg_cols = ['priceineur', 'feeineur', 'winningsineur']
for c in not_neg_cols:
    print(f"Records with negative {c} value: {df.where(col(c) < 0).count()}")


Dataframe shape: (5102610, 11)
Records with empty timestamp value: 0
Records with empty sitetid value: 249097
Records with empty customernumber value: 100216
Records with empty gamename value: 199006
Records with empty priceineur value: 199006
Records with empty feeineur value: 199006
Records with empty ticketexternalid value: 50600
Records with negative priceineur value: 0
Records with negative feeineur value: 0
Records with negative winningsineur value: 0


### Lottery games data overview:

Import the data and show the first 10 rows:

In [14]:
df = spark.read.csv(lotterygames_file, header=True, sep=';')

In [15]:
df.show(n=10)

+-------------+--------+--------------+--------+-------------+----------------+------------+---------------+--------------------+--------+--------+--------------------+
|timestampunix|    site|customernumber|currency|amountincents|feeamountincents|        game|orderidentifier|paymentamountincents|ticketid|betindex|            discount|
+-------------+--------+--------------+--------+-------------+----------------+------------+---------------+--------------------+--------+--------+--------------------+
|   1558931809|websiteA|        132197|     eur|          600|              60|       LOTTO|        ZAH1258|                null|   34771|      12|{"discountInMinor...|
|   1544542681|websiteE|        131323|     eur|          750|              75| CASH_4_LIFE|        HAC4670|                null|   93454|       3|{"discountInMinor...|
|   1577412253|websiteH|        130749|     eur|          750|              75| CASH_4_LIFE|        DEJ1680|                null|   02341|       3|{"discou

In [16]:
print(f"Dataframe shape: ({df.count()}, {len(df.columns)})")
not_null_cols =  ['timestampunix', 'site', 'customernumber', 'amountincents',
                  'feeamountincents', 'game', 'orderidentifier', 'ticketid']
for c in not_null_cols:
    print(f"Records with empty {c} value: {df.where(col(c).isNull()).count()}")

not_neg_cols = ['amountincents', 'feeamountincents', 'paymentamountincents']
for c in not_neg_cols:
    print(f"Records with negative {c} value: {df.where(col(c) < 0).count()}")

Dataframe shape: (5099164, 12)
Records with empty timestampunix value: 248768
Records with empty site value: 201642
Records with empty customernumber value: 100844
Records with empty amountincents value: 199937
Records with empty feeamountincents value: 199937
Records with empty game value: 50947
Records with empty orderidentifier value: 50795
Records with empty ticketid value: 50980
Records with negative amountincents value: 0
Records with negative feeamountincents value: 0
Records with negative paymentamountincents value: 0
